In [1]:
import yaml
import seaborn as sns
import pandas as pd
import copy

In [2]:
from utils.helper import * 
from models.build_model import *

In [3]:
def remove_nano_from_input(arr, feature_names):
    for i in range(len(feature_names)):
        if "nano" in feature_names[i]:
            arr[:, i] = 0
    return arr

In [4]:
ann_config_path = 'configs/ann.yaml'
with open(ann_config_path, "r") as f:
    config = yaml.safe_load(f)
k_fold = config["k_fold"]
save_path = config["Dataset"]["save_path"]
model = build_model(config)
model.load()

for i in range(k_fold):
    xtrain, ytrain, xtest, ytest = model.inputs[i]
    feature_names = model.feature_names
    pred = model.models[i].predict(xtest)
    
    xtest_nonano = remove_nano_from_input(copy.deepcopy(xtest), feature_names)
    pred_nonano = model.models[i].predict(xtest_nonano)
    
    fig = plt.figure(figsize=(10,10))
    plt.plot(ytest, ytest, "k--")
    plt.scatter(ytest, pred, marker="*", color='blue')
    plt.scatter(ytest, pred_nonano, marker="+", color='red')
    
    plt.tight_layout()

    plt.savefig(f"{save_path}/wo_nano_scatter_{i}_fold.png")
    plt.close("all")

In [5]:
rf_config_path = 'configs/random_forest.yaml'
with open(rf_config_path, "r") as f:
    config = yaml.safe_load(f)
k_fold = config["k_fold"]
save_path = config["Dataset"]["save_path"]
model = build_model(config)
model.load()

for i in range(k_fold):
    xtrain, ytrain, xtest, ytest = model.inputs[i]
    feature_names = model.feature_names
    pred = model.models[i].predict(xtest)
    
    xtest_nonano = remove_nano_from_input(copy.deepcopy(xtest), feature_names)
    pred_nonano = model.models[i].predict(xtest_nonano)
    
    fig = plt.figure(figsize=(10,10))
    plt.plot(ytest, ytest, "k--")
    plt.scatter(ytest, pred, marker="*", color='blue')
    plt.scatter(ytest, pred_nonano, marker="+", color='red')
    
    plt.tight_layout()

    plt.savefig(f"{save_path}/wo_nano_scatter_{i}_fold.png")
    plt.close("all")

In [6]:
xgb_config_path = 'configs/xgboost.yaml'
with open(xgb_config_path, "r") as f:
    config = yaml.safe_load(f)
k_fold = config["k_fold"]
save_path = config["Dataset"]["save_path"]
model = build_model(config)
model.load()

for i in range(k_fold):
    xtrain, ytrain, xtest, ytest = model.inputs[i]
    feature_names = model.feature_names
    pred = model.models[i].predict(xtest)
    
    xtest_nonano = remove_nano_from_input(copy.deepcopy(xtest), feature_names)
    pred_nonano = model.models[i].predict(xtest_nonano)
    
    fig = plt.figure(figsize=(10,10))
    plt.plot(ytest, ytest, "k--")
    plt.scatter(ytest, pred, marker="*", color='blue')
    plt.scatter(ytest, pred_nonano, marker="+", color='red')
    
    plt.tight_layout()

    plt.savefig(f"{save_path}/wo_nano_scatter_{i}_fold.png")
    plt.close("all")

In [7]:
ridge_config_path = 'configs/ridge_regression.yaml'
with open(ridge_config_path, "r") as f:
    config = yaml.safe_load(f)
k_fold = config["k_fold"]
save_path = config["Dataset"]["save_path"]
model = build_model(config)
model.load()

for i in range(k_fold):
    xtrain, ytrain, xtest, ytest = model.inputs[i]
    feature_names = model.feature_names
    pred = model.models[i].predict(xtest)
    
    xtest_nonano = remove_nano_from_input(copy.deepcopy(xtest), feature_names)
    pred_nonano = model.models[i].predict(xtest_nonano)
    
    fig = plt.figure(figsize=(10,10))
    plt.plot(ytest, ytest, "k--")
    plt.scatter(ytest, pred, marker="*", color='blue')
    plt.scatter(ytest, pred_nonano, marker="+", color='red')
    
    plt.tight_layout()

    plt.savefig(f"{save_path}/wo_nano_scatter_{i}_fold.png")
    plt.close("all")